In [8]:
#!pip install openai --upgrade
#!pip install langchain --upgrade
#!pip install llama-index --upgrade
#!pip install python-dotenv

This script takes a jenkins log, uses it to augment the gpt model, then asks gpt to summarize errors and then recursively gives information about terms mentioned in the errors. These responses are combined and given as a log error summarization.  

In [38]:
import os
import openai
from dotenv import load_dotenv
from llama_index import GPTListIndex, SimpleDirectoryReader, PromptHelper, LangchainEmbedding, ServiceContext
from llama_index.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [39]:


# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_version = os.getenv('OPENAI_API_VERSION')
# openai.log = "debug"

In [79]:


# Initialize LLM and Embeddings model (model is the actual model name, e.g., gpt-35-turbo, engine is your custom deployment name, e.g., my-gpt-35-turbo)
llm = AzureOpenAI(engine="loveadagpt3516k_2", model="gpt-35-turbo-16k", temperature=0.0)
#embeddings = LangchainEmbedding(OpenAIEmbeddings(deployment_id="loveadaembed2", chunk_size=64))

# Define prompt helper
#prompt_helper = PromptHelper(context_window=3000, 
#                             num_output=500, 
#                             chunk_overlap_ratio=0.1, 
#                             chunk_size_limit=1000)

service_context = ServiceContext.from_defaults(llm=llm)#, embed_model=embeddings)#, prompt_helper=prompt_helper)

# Load documents
documents = SimpleDirectoryReader('../data/').load_data()

# Create index
index = GPTListIndex.from_documents(documents)#, service_context=service_context, prompt_helper=prompt_helper)
query_engine = index.as_query_engine(response_mode='tree_summarize',service_context=service_context, verbose=True)



In [81]:
query = input()
answer = query_engine.query(query)
print(f"Query: {query}")
print(f"Answer: {answer}")
print(f"Sources: {answer.get_formatted_sources()}")

InvalidRequestError: The response was filtered due to the prompt triggering Azure OpenAI’s content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766

In [ ]:
#comparison model
